In [1]:
import tensorflow as tf
from dataset_tf import VoxDataset, DataGenerator
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
dataset = VoxDataset()

In [3]:
len(dataset)

107640

In [4]:
DATASET_SIZE = len(dataset)

In [5]:
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = DATASET_SIZE - train_size - val_size

In [6]:
print(train_size)
print(test_size)
print(val_size)

75348
16146
16146


In [7]:
train_indices = np.random.choice(DATASET_SIZE, size=train_size, replace=False).tolist()
val_indices = np.random.choice(list(set(range(DATASET_SIZE)) - set(train_indices)), size=val_size, replace=False).tolist()
test_indices = list(set(range(DATASET_SIZE)) - set(train_indices) - set(val_indices))


In [8]:
train = VoxDataset(indices=train_indices)
test = VoxDataset(indices=test_indices)
val = VoxDataset(indices=val_indices)

In [9]:
type(train_indices)

list

In [10]:
print(len(train))
print(len(test))
print(len(val))

75348
16146
16146


In [11]:
train = DataGenerator(train, batch_size=32)
test = DataGenerator(test, batch_size=1)
val = DataGenerator(val, batch_size=32)

In [12]:
dataset[0][0].shape

TensorShape([384])

In [13]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
len(dataset[0][0])

384

In [15]:
import tensorflow as tf

def AutoEncoder():
    # Define input layer
    input_layer = tf.keras.layers.Input(shape=(384,))
    
    # Encoder layers
    # encoder = tf.keras.layers.Dense(192, activation='tanh')(input_layer)
    
    # Decoder layers
    decoder = tf.keras.layers.Dense(500, activation='tanh')(input_layer)
    decoder = tf.keras.layers.Dense(768, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(1000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(1536, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(2000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(3072, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(5000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(6144, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(8000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(10000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(12288, activation='tanh')(decoder)
    decoder_output = tf.keras.layers.Dense(15069, activation='tanh')(decoder)
    
    # Define the model
    model = tf.keras.models.Model(inputs=input_layer, outputs=decoder_output)
    
    return model




In [16]:
model = AutoEncoder()


In [17]:
loss_fn = tf.keras.losses.MeanSquaredError()

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Define the number of epochs
num_epochs = 10

# Train the model using model.fit
model.fit(train, epochs=10, batch_size=32, validation_data=val)

Epoch 1/10


  11/2355 [..............................] - ETA: 1:35:37 - loss: 0.1000 - accuracy: 0.0085

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense (Dense)               (None, 500)               192500    
                                                                 
 dense_1 (Dense)             (None, 768)               384768    
                                                                 
 dense_2 (Dense)             (None, 1000)              769000    
                                                                 
 dense_3 (Dense)             (None, 1536)              1537536   
                                                                 
 dense_4 (Dense)             (None, 2000)              3074000   
                                                                 
 dense_5 (Dense)             (None, 3072)              614707

In [ ]:
loss, acc = model.evaluate(test)

150/150 [==============================] - 11s 71ms/step - loss: 4.9943e-04 - accuracy: 0.8267


In [ ]:
print(f"Accuracy: {acc*100:.4f}%\nLoss: {loss:.4f}")

Accuracy: 82.6667%
Loss: 0.0005


In [ ]:
y_pred = []

In [ ]:
for i in tqdm(test):
    pred = model.predict(i[0], verbose=0)*139.8999
    y_pred.append(VoxDataset.to_mesh_points(pred))



100%|██████████| 150/150 [00:17<00:00,  8.81it/s]


In [ ]:
import open3d as o3d
import numpy as np

for index, i in enumerate(y_pred):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(i)
    # o3d.visualization.draw_geometries([pcd])
    o3d.io.write_point_cloud(f"./Test/data{index}.ply", pcd)